Text Summarisation using Pointer Generation Networks

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Add
from tensorflow.keras.models import Model


In [4]:
vocab_size = 50000  # Adjust based on your dataset
embedding_dim = 256
hidden_units = 512

In [6]:
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]


In [8]:
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

In [10]:
pointer_gen_output = Dense(vocab_size, activation='softmax')(decoder_outputs)
gen_output = Dense(vocab_size, activation='softmax')(decoder_outputs)

In [18]:
coverage_vector = Input(shape=(None, hidden_units))  # Shape should match LSTM output
coverage_dense = Dense(hidden_units, activation='relu')(coverage_vector)

In [20]:
coverage_projected = Dense(vocab_size)(coverage_dense)


In [24]:
coverage_output = Add()([pointer_gen_output, coverage_projected])


In [26]:
model = Model(inputs=[encoder_inputs, decoder_inputs, coverage_vector], outputs=coverage_output)


In [28]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [30]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, None, 256)         │      12,800,000 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, None, 256)         │      12,800,000 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, None, 512),       │       1,574,912 │ embedding[0][0]            │
│                               │ (None, 512), (None, 512)] │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_4 (InputLayer)    │ (None, None, 512)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, None, 512),       │       1,574,912 │ embedding_1[0][0],         │
│                               │ (None, 512), (None, 512)] │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, None, 512)         │         262,656 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, None, 50000)       │      25,650,000 │ lstm_1[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, None, 50000)       │      25,650,000 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_2 (Add)                   │ (None, None, 50000)       │               0 │ dense[0][0], dense_5[0][0] │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 80,312,480 (306.37 MB)

 Trainable params: 80,312,480 (306.37 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [34]:
num_epochs = 10
batch_size = 64
max_encoder_seq_length = 100  # Adjust based on your dataset
max_decoder_seq_length = 100 

In [36]:
encoder_input_data = np.random.randint(0, 50000, size=(1000, max_encoder_seq_length))
decoder_input_data = np.random.randint(0, 50000, size=(1000, max_decoder_seq_length))
coverage_vector_data = np.random.rand(1000, max_decoder_seq_length, hidden_units)
target_data = np.random.randint(0, 50000, size=(1000, max_decoder_seq_length))


In [38]:
encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_decoder_seq_length, padding='post')
coverage_vector_data = pad_sequences(coverage_vector_data, maxlen=max_decoder_seq_length, padding='post')
target_data = pad_sequences(target_data, maxlen=max_decoder_seq_length, padding='post')

In [40]:
model.fit(
    x=[encoder_input_data, decoder_input_data, coverage_vector_data], 
    y=target_data, 
    epochs=num_epochs, 
    batch_size=batch_size
)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall/gradient_tape/functional_1/dense_5_1/MatMul/MatMul_1 defined at (most recent call last):
<stack traces unavailable>
OOM when allocating tensor with shape[64,512,50000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node StatefulPartitionedCall/gradient_tape/functional_1/dense_5_1/MatMul/MatMul_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_one_step_on_iterator_5013]